In [102]:
# Justin Harper
# WSU ID: 10696738

## Static Scoping Assignment (SSPS)
### Cpts 355 - Spring 2016

### Assigned April 4, 2016

### Due Monday, April 18, 2016
Continue to develop Static Scoped Post Script Interperter


## Your Code Start Here

In [103]:
# The operand stack: define the operand stack and its operations in this notebook cell
opstack = []
# other globals for me to use
var = ''
brace = ''
curly = False


# now define functions to push and pop values on the opstack according to your decision about which
# end should be the hot end. Recall that `pass` in python is a no-op: replace it with your code.

def opPop(): 
    return opstack.pop()

def opPush(value):
    opstack.append(value)

# Remember that there is a Postscript operator called "pop" so we choose different names for these functions.


In [104]:
# The dictionary stack: define the dictionary stack and its operations in this cell
dstack = [{}]

# now define functions to push and pop dictionaries on the dictstack, to define name, and to lookup a name

def dictPop():
    a = dstack.pop()
    if(a == {}):
        dictPush(a)
        return {}
    else:
        return a
    

def dictPush(value):
    dstack.append(value)

def define(name, value):
    a = opstack.pop()
    b = opstack.pop()
    x = dstack.pop()
    x[b] = a
    dstack.append(x)

def lookup(name):
    # return the value associated with name
    # what is your design decision about what to do when there is no definition for name
    
    # need to determine if name is a number or a name
    #if name.isdigit():
    #    return name
    global mustsearch
    global ind
    
    if(ind >= 1):
        uindD = dstack.pop()
        indirectD = dstack.pop()
        ind -= 1
        result = lookup(name)
        dstack.append(indirectD)
        dstack.append(uindD)
        return result   
    
    AD = dstack.pop()
    if name in AD.keys():
        if(switch == 'static' and mustsearch == True):
            int += 1
        result = AD[name]
    elif (dstack and switch == 'static'):
        mustsearch = True
        result = lookup(name)
    else:
        result = name
    dstack.append(AD)
    return result    
    
        

In [105]:
# Arithmetic operators: define all the arithmetic operators in this cell -- add, sub, mul, div, eq, lt, gt

def Equal():
    lookup(opstack.pop())


def Eq():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    
    if(a == b):
        opstack.append("true")
    else:
        opstack.append("false")

def Add():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    sum = int(a) + int(b)
    opstack.append(sum)
    
def Mul():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    prod = int(a) * int(b)
    opstack.append(prod)

def Div():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    
    if(a == 0):
        exit()
    else:
        quote = int(b) / int(a)
        opstack.append(quote)
    

def Lt():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    
    if(b < a):
        opstack.append("true")
    else:
        opstack.append("false")

def Gt():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    
    if(b > a):
        opstack.append("true")
    else:
        opstack.append("false")


In [106]:
# Boolean operators: define all the boolean operators in this cell -- and, or, not
def And():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    
    if(a == 'true' and b == 'true'):
        opstack.append('true')
    else:
        opstack.append('false')
        
def Not():
    a = lookup(opstack.pop())
    if(a == 'false'):
        opstack.append('true')
    else:
        opstack.append('false')
        
def Or():
    a = lookup(opstack.pop())
    b = lookup(opstack.pop())
    
    if(a == 'true' or b == 'true'):
        opstack.append('true')
    else:
        opstack.append('false')

In [107]:
# Define the stack manipulation operators in this cell: dup, exch, pop
def Dup():
    a = opstack.pop()
    opstack.append(a)
    opstack.append(a)
    
def Exch():
    a = opstack.pop()
    b = opstack.pop()
    
    opstack.append(a)
    opstack.append(b)
def Pop():
    opstack.pop()

In [108]:
# Define the dictionary manipulation operators in this cell: dict, begin, end, def
# name the function for the def operator psDef because def is reserved in Python
def Dict():
    # takes one operand from the operand stack, ignores it, and creates a new, empty dictionary on the operand stack
    opstack.pop()
    dstack.append({})
    
def Begin(d):
    # begin pushes a dictionary on the dictionary stack
    dstack.append(d)
def End():
    #  end removes a dictionary from the dictionary stack
    dstack.pop()
    
def Def():
    # def creates or modifies an entry in the top dictionary on the dictionary stack.
    a = opstack.pop()
    b = opstack.pop()
    d = dstack.pop()
    d[str(b)] = a
    dstack.append(d)
    

In [109]:
# Define the IF ELSE and ELSEIF operators
# ELSE will be handled in processLine

def If():
    l = opstack.pop()
    b = opstack.pop()
    if b == 'true':
        processLine(l[1:-1])
        
        
def ElseIf():
    lf = opstack.pop()
    lt = opstack.pop()
    b = opstack.pop()
    
    if b == 'true':
        processLine(lt[1:-1])
    else:
        processLine(lf[1:-1])
        
    
    


In [110]:
# Define processLine
def processLine(line):
    global brace
    global var
    global curly
    
    if(switch == 'static'):
        dstack.append({})
    
    
    for c in line.split():
        if c[0] >='0' and c[0] <= '9':
            opstack.append(int(c))
        elif c == '{':
            brace = c
            curly = True
        elif curly:
            brace +=' '
            brace += c
            if  c == '}':
                curly = False
                opstack.append(brace)
        elif c[0] == '/':
            var = c[1:]
            opstack.append(var)
        
        else:
            handleIt(c)
    if(switch == 'static'):
        dstack.pop()

In [111]:
# Define the printing operators in this cell: =, stack
# Pick a good name for the code implementing =
def PrintOP():
    print(opstack)

def PrintD():
    print(dstack)
    
def PrintVal():
    a = lookup(opstack.pop())
    print (a)

In [112]:
# Define handleIt()

def handleIt(token):
    print(token)
    if token == '=': Equal()
    #do logical stuff
    elif token == 'and': And()
    elif token == 'or' : Or()
    elif token == 'eq' : Eq()
    elif token == 'not': Not()
    elif token == 'gt' : Gt()
    elif token == 'lt' : Lt()
    # do aritmatic stuff
    elif token == 'add': Add()
    elif token == 'sub': Sub()
    elif token == 'mul': Mul()
    elif token == 'div': Div()
        
    #not logical or math must be something else
    elif token == 'stack': PrintOP()
    elif token == 'if': If()
    elif token == 'ifelse': ElseIf()
    elif token == 'begin': Begin()
    elif token == 'dup': Dup()
    elif token == 'exch': Exch()
    elif token == 'pop': Pop()
    elif token == 'def': Def()
    elif token == 'end': End()
        
    else:
        if (switch == 'static'):
            mustsearch = False
        s = lookup(str(token))
        print("s = " + s)
        if(s[0] == '{'):
            processLine(s[1:-1])
        else:
            opstack.append(s)

In [114]:
#****************Entry point ***********************#
global switch
global mustsearch
global ind

ind = 0

switch = 'static' # change to 'dynamic' for old mode (just need to make it not 'static')


PostScript = ['/x 4 def', '/g { x stack } def', '/f { /x 7 def g } def', 'f'] # maybe this works?


PS = [PostScript]

for x in PostScript:
    print(x)
    processLine(x)
    
    
print('======STACK======')
for x in opstack:
    print(x)
print('======DICT======')
for x in dstack:
    print('----', x)
    
    



/x 4 def
def
/g { x stack } def
def
/f { /x 7 def g } def
def
f
f
s = f
======STACK======
f
f
f
f
======DICT======
---- {}


## Test your code
With all of that stuff defined, you will be able to test your interpreter using Python code like this: